In [1]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 4.5 MB/s eta 0:00:00


In [2]:
import praw

In [4]:
reddit = praw.Reddit(client_id = "JgHZRyaH_Ya0sSPJ4WB4ZA", client_secret="yNybsXSQCZVR-r-2mo0Ribuae-zKgw", user_agent="scrape")

In [5]:
subreddit_name = "IAmA"

In [6]:
subreddit = reddit.subreddit(subreddit_name)